### Harmonic distance calculation

In [1]:
import pandas as pd
from sklearn.metrics import pairwise_distances

#### Spotify mapping of key index to key

In [2]:
note_table = [ 'C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Bb', 'B' ]

#### Circle of fifths
According to harmonic progression, nearby keys are harmonic. Because it's a circle, 'C' is also next to 'F'.

In [3]:
fifth_table = [ 7 * i % 12 for i in range(12) ]
[(i, note_table[i]) for i in fifth_table]

[(0, 'C'),
 (7, 'G'),
 (2, 'D'),
 (9, 'A'),
 (4, 'E'),
 (11, 'B'),
 (6, 'F#/Gb'),
 (1, 'C#/Db'),
 (8, 'G#/Ab'),
 (3, 'D#/Eb'),
 (10, 'A#/Bb'),
 (5, 'F')]

#### Mapping of distance between keys to harmonic distance
That is if the distance in key index is 7, e.g. 'C' -> 'G', the distance is only 1 because it is just one fifth above. Or, the distance from 'D' to 'C' is 2 which also happens to be two fifths (downwards).

In [4]:
inv_fifth_table = [ (fifth_table.index(i)+5)%12 - 5 for i in range(12) ]
inv_fifth_table

[0, -5, 2, -3, 4, -1, 6, 1, -4, 3, -2, 5]

#### Distance-function for comparing keys and a distance table between all keys

In [5]:
def key_distance(key1: int, key2: int):
  return inv_fifth_table[(key2 - key1) % 12]
pd.DataFrame(
  [ [ key_distance(i, j) for j in range(12) ] for i in range(12) ],
  index = note_table,
  columns = note_table)

,C,C#/Db,D,D#/Eb,E,F,F#/Gb,G,G#/Ab,A,A#/Bb,B
C,0,-5,2,-3,4,-1,6,1,-4,3,-2,5
C#/Db,5,0,-5,2,-3,4,-1,6,1,-4,3,-2
D,-2,5,0,-5,2,-3,4,-1,6,1,-4,3
D#/Eb,3,-2,5,0,-5,2,-3,4,-1,6,1,-4
E,-4,3,-2,5,0,-5,2,-3,4,-1,6,1
F,1,-4,3,-2,5,0,-5,2,-3,4,-1,6
F#/Gb,6,1,-4,3,-2,5,0,-5,2,-3,4,-1
G,-1,6,1,-4,3,-2,5,0,-5,2,-3,4
G#/Ab,4,-1,6,1,-4,3,-2,5,0,-5,2,-3
A,-3,4,-1,6,1,-4,3,-2,5,0,-5,2


#### Metric that calculates euclidean distance but aware of the cyclic key distance:

In [6]:
def key_aware_metric(X, Y, key_index=None, mode_index=None):
  if key_index == None:
    return (sum((X - Y)**2))**.5
  
  ordinary_columns = [ i for i in range(len(X)) if i != key_index ] # still take mode as an ordinary column, too
  sum2 = sum((X[ordinary_columns] - Y[ordinary_columns])**2)
  key_x = int(X[key_index])
  key_y = int(Y[key_index])
  if mode_index != None:
    key_x += 3 if X[mode_index] == 0 else 0
    key_y += 3 if Y[mode_index] == 0 else 0
  sum2 += key_distance(key_x, key_y)**2
  return sum2**.5

#### And a variant of `pairwise_distances` that recognizes the `key` column and applies the `key_aware_metric`:

In [7]:
def key_aware_pairwise_distances(df):
  try:
    key_index = df.columns.values.tolist().index('key')
  except:
    key_index = None
  try:
    mode_index = df.columns.values.tolist().index('mode')
  except:
    mode_index = None
    
  return pairwise_distances(df, metric=key_aware_metric, key_index=key_index, mode_index=mode_index)

#### Example:

In [88]:
df = (
  pd.DataFrame([
      [ f'Song in {key} {"major" if mode else "minor"}', note_table.index(key), mode] 
      for key, mode in [('C', 1), ('C', 0), ('C#/Db', 1), ('B', 1), ('G', 1), ('E', 0)]
    ],
    columns=['song', 'key', 'mode'])
  .set_index('song')
  .sort_values('song')
)
df

,key,mode
song,,
Song in B major,11,1
Song in C major,0,1
Song in C minor,0,0
Song in C#/Db major,1,1
Song in E minor,4,0
Song in G major,7,1


Notice:
- how 'C major' and 'C# major' are considered close 
- while 'G major' is considered far away even those it is the dominant chord to 'C major'
- 'B major' and 'C major' are in contrast not considered close because wrapping the scale is not implemented
- 'G major' and 'E minor' are considered far apart even though, they are parallel keys
- 'C major' and 'C minor' are considered close, even though harmonically they are far apart because 'C minor' has parallel 'D# major' which is not close

In [89]:
pd.DataFrame(
  pairwise_distances(df),
  index=df.index,
  columns=df.index)  

song,Song in B major,Song in C major,Song in C minor,Song in C#/Db major,Song in E minor,Song in G major
song,,,,,,
Song in B major,0.000000,11.000000,11.045361,10.000000,7.071068,4.000000
Song in C major,11.000000,0.000000,1.000000,1.000000,4.123106,7.000000
Song in C minor,11.045361,1.000000,0.000000,1.414214,4.000000,7.071068
Song in C#/Db major,10.000000,1.000000,1.414214,0.000000,3.162278,6.000000
Song in E minor,7.071068,4.123106,4.000000,3.162278,0.000000,3.162278
Song in G major,4.000000,7.000000,7.071068,6.000000,3.162278,0.000000


In contrast, notice now:
- how 'C major' and 'C# major' are now considered harmonically far apart while 
- 'C major' and 'G major' are close
- 'C major' and 'B major' are now equally far apart as 'C major' and 'C# major' which are both a single half-tone apart, i.e. scale wrapping works
- 'G major' and 'E minor' are now close and only differ by the mode difference, not a key difference anymore
- 'C major' and 'C minor' are now in fact far apart

In [90]:
pd.DataFrame(
  key_aware_pairwise_distances(df),
  index=df.index,
  columns=df.index)

song,Song in B major,Song in C major,Song in C minor,Song in C#/Db major,Song in E minor,Song in G major
song,,,,,,
Song in B major,0.000000,5.000000,4.123106,2.000000,4.123106,4.000000
Song in C major,5.000000,0.000000,3.162278,5.000000,1.414214,1.000000
Song in C minor,4.123106,3.162278,0.000000,2.236068,4.000000,4.123106
Song in C#/Db major,2.000000,5.000000,2.236068,0.000000,6.082763,6.000000
Song in E minor,4.123106,1.414214,4.000000,6.082763,0.000000,1.000000
Song in G major,4.000000,1.000000,4.123106,6.000000,1.000000,0.000000


# Harmonic unwrapping

This approach takes the (key, mode) columns and transforms the key into an index in the circle of fifths such that then, indeed neighboring values are harmonically related. It also uses the mode to transform minor keys into the corresponding major parallel key. Optionally, it can duplicate rows an octave above such that Euclidean distance in k-means has a chance to find related songs even if the key is close across the circle of fifths split-point (F#/Gb).

In [91]:
def harmonic_scale(df, unwrap=True, drop_extra=True):
  '''
  Transform (key, mode) columns into (harmonic, mode) columns in which the new "harmonic"
  column has the property that euclidean distance means harmonic distance, i.e. C major and
  G major are next to each other. In order to also relate keys across the F#/Gb to C#/Db 
  break point of the circle of fifths, each row is duplicated and tranposed an octave up 
  optionally.
  
  :param df: original data-frame, must have `key` and `mode` columns
  :param unwrap: whether to create duplicated rows transposed an octave up
  :param drop_extra: whether to drop extra columns major_key and key after transformation
  :return: new data-frame
  '''
  df = (
    df
    .assign(major_key=lambda x: (x.key + (1-x['mode']) * 3)%12)
    .assign(harmonic=lambda x: x.apply(lambda r: inv_fifth_table[r.major_key], axis=1))
  )
  if unwrap:
    df = pd.concat(
      [ 
        df.iloc[[row]].assign(harmonic=lambda x: x.harmonic+shift*12) 
        for row in range(len(df)) 
        for shift in range(0,2) 
      ]
    )
  if drop_extra:
    df = df.drop(['major_key', 'key'], axis=1)
  return df

## Example

In [92]:
harmonic_df = harmonic_scale(df)
harmonic_df

,mode,harmonic
song,,
Song in B major,1,5
Song in B major,1,17
Song in C major,1,0
Song in C major,1,12
Song in C minor,0,-3
Song in C minor,0,9
Song in C#/Db major,1,-5
Song in C#/Db major,1,7
Song in E minor,0,1


In [93]:
(
  pd.DataFrame(
    pairwise_distances(harmonic_df[['harmonic', 'mode']]),
    index=harmonic_df.index,
    columns=harmonic_df.index)
 .groupby(harmonic_df.index.name)
 .min()
 .T
 .groupby(harmonic_df.index.name)
 .min()
)

song,Song in B major,Song in C major,Song in C minor,Song in C#/Db major,Song in E minor,Song in G major
song,,,,,,
Song in B major,0.000000,5.000000,4.123106,2.000000,4.123106,4.000000
Song in C major,5.000000,0.000000,3.162278,5.000000,1.414214,1.000000
Song in C minor,4.123106,3.162278,0.000000,2.236068,4.000000,4.123106
Song in C#/Db major,2.000000,5.000000,2.236068,0.000000,6.082763,6.000000
Song in E minor,4.123106,1.414214,4.000000,6.082763,0.000000,1.000000
Song in G major,4.000000,1.000000,4.123106,6.000000,1.000000,0.000000


This is the same result as the key-aware metric above!